# Collisions

### **Introduction**

A collision is an event in which two or more objects come into contact with each other with a significant force. These interactions often involve a transfer of energy and momentum. Collisions are a common phenomenon in many fields of physics and engineering, from understanding the behavior of particles in a gas to analyzing the dynamics of vehicles during car accidents. The outcome of a collision depends on factors like the type of collision, the masses of the objects involved, and their velocities.

### **What is it?**

In physics, a collision can be described as an event where two or more objects exert forces on each other for a brief period. The key characteristics of a collision are:

- **Momentum Conservation:** In a closed system (no external forces), the total momentum of the system before the collision is equal to the total momentum after the collision.
- **Energy Conservation:** Depending on the type of collision, the total mechanical energy (kinetic energy) may be conserved or converted into other forms of energy, such as heat or sound.

Collisions can be classified into two main types:

- **Elastic Collision:** In this type of collision, both momentum and kinetic energy are conserved. The objects rebound off each other without any loss in the total kinetic energy.
- **Inelastic Collision:** In an inelastic collision, momentum is conserved, but kinetic energy is not. Some of the kinetic energy is converted into other forms of energy, such as heat or deformation of the objects.
Perfectly Inelastic Collision: This is a special case of inelastic collision where the objects stick together after the collision, and the maximum amount of kinetic energy is lost.

### **Example**

Consider two billiard balls colliding on a pool table:
- Ball A has a mass of 0.17 kg and is **moving at 2 m/s**.
- Ball B has a mass of 0.17 kg and is **stationary**.

In an elastic collision, the balls would rebound off each other. Since momentum is conserved, the velocity of Ball A will change, and Ball B will start moving after the collision. The kinetic energy before and after the collision would remain the same.

In an inelastic collision, some of the kinetic energy will be lost in the form of sound or heat. The balls may still bounce off each other, but their velocities will not change as much as in an elastic collision, and energy will be dissipated.

In a perfectly inelastic collision, the balls would stick together after the collision, moving as one object with a combined mass. Here, kinetic energy is not conserved (it is lost), but momentum is.

The behavior of the system will vary depending on whether the collision is elastic or inelastic, illustrating how different types of collisions affect the dynamics of the objects involved.


# Simulation

## Setup

Installing packages (for Google Colab). If this notebook is opened in Google Colab then some packages must be installed to run the code!\
Then importing the scene and plot settings.

In [ ]:
#@title Run to install MuJoCo and `dm_control` for Google Colab

IS_COLAB = 'google.colab' in str(get_ipython())
if IS_COLAB:
    # download the repository
    !git clone https://github.com/commanderxa/alphalabs.git
    from alphalabs.mechanics.setup import install_packages_colab
    install_packages_colab()
    import alphalabs.mechanics.plot
    from alphalabs.mechanics.scene import Scene
else:
    import os, sys
    module_path = os.path.abspath(os.path.join(".."))
    if module_path not in sys.path:
        sys.path.append(module_path)
    # import the scene
    from scene import Scene
    import plot

## Import

Import all required packages to preform simulations. Packages include simulation engine, plotting libraries and other ones necessary for computations.

In [ ]:
%env MUJOCO_GL=egl

import os

# simulation
from dm_control import mjcf

# for video recording
import mediapy

# computations
import numpy as np

# plot charts
import seaborn as sns
import matplotlib.pyplot as plt

## Initial Conditions

In this block constants are defined. They impact the environment, rendering and objects directly.

**Note**, don't set very high values for velocity as the simulation might crash. If you will experience such a situation, try reducing the velocity!

In [ ]:
# global
viscosity = 0.00002 # Air Resistance (this is default value for air on Earth)

# collision constants
distance = 1 # distance between two objects (center points of object) [meters]
# mass [kg]
left_mass = 0.25 # mass of left box
right_mass = 0.25 # mass of right box
# speed [m/s]
left_velocity = 1.2
right_velocity = 0

"""
Collision Type of Simulation

set to 'True' to simulate 'inelastic'
set to 'False' to simulate 'elastic'
"""
is_inelastic = False

# rendering
width = 1920
height = 1080
dpi = 600
duration = 5  # (seconds)
framerate = 60  # (Hz)

## Model

### Object

This class defines the object of our interest, a `box`. Here we write what is this object (box), what can it do (move, fall) and also add a camera that follows the object.

In [ ]:
class Box(object):

    def __init__(
        self, side: str, rgba: list[float], mass: float, is_inelastic: bool
    ) -> None:
        self.model = mjcf.RootElement(model=f"{side}_box")

        self.box = self.model.worldbody.add("body", name="box", pos=[0, 0, 0])

        # self.box.add("inertial", pos=[0, 0, 0], mass=mass)

        solref = {}
        if not is_inelastic:
            solref["solref"] = [0.02, 0.07]

        self.box_geom = self.box.add(
            "geom",
            name="box_geom",
            type="box",
            pos=[0, 0, 0],
            size=[0.1, 0.05, 0.04],
            mass=mass,
            rgba=rgba,
            condim=1,
            **solref,
        )

        self.move = self.box.add("joint", name="move", type="slide", axis=[1, 0, 0])
        self.fall = self.box.add("joint", name="fall", type="slide", axis=[0, 0, 1])

        if is_inelastic:
            self.model.actuator.add(
                "adhesion",
                name=f"{side}_adhesion",
                body="box",
                ctrlrange=[0.99, 1],
                gain=2.5,
            )

### World Model

Collecting everything into one general model.

In [ ]:
class Model(object):

    def __init__(self, right_mass: float, left_mass: float) -> None:
        self.model = mjcf.RootElement(model="model")

        # setting environment constants
        self.model.option.viscosity = viscosity
        self.model.option.integrator = "RK4"
        self.model.option.flag.constraint = "enable"
        self.model.option.flag.contact = "enable"
        self.model.option.flag.gravity = "enable"
        self.model.option.flag.energy = "enable"

        # set render info
        self.model.visual.__getattr__("global").offheight = height
        self.model.visual.__getattr__("global").offwidth = width

        # create the environment (ground)
        self.arena = Scene(length=10)
        self.arena_site = self.model.worldbody.add(
            "site", name="arena_site", pos=[0, 0, -0.1], rgba=[0, 0, 0, 0]
        )
        self.arena_site.attach(self.arena.model)

        # add object1
        self.box1 = Box(
            side="left", rgba=[0, 0, 1, 1], mass=left_mass, is_inelastic=is_inelastic
        )
        box_site1 = self.model.worldbody.add(
            "site", name="left_site", pos=[-distance / 2 - 0.05, 0, 0.02], rgba=[0, 0, 0, 0]
        )
        box_site1.attach(self.box1.model)

        # add object2
        self.box2 = Box(
            side="right", rgba=[1, 0, 0, 1], mass=right_mass, is_inelastic=is_inelastic
        )
        box_site2 = self.model.worldbody.add(
            "site", name="right_site", pos=[distance / 2 + 0.05, 0, 0.02], rgba=[0, 0, 0, 0]
        )
        box_site2.attach(self.box2.model)

        self.camera = self.model.worldbody.add(
            "camera",
            name="front",
            pos=[0, -2, 1],
            euler=[60, 0, 0],
        )

## Simulation

Initializing the `physics` of the simulation.

In [ ]:
model = Model(right_mass, left_mass).model
physics = mjcf.Physics.from_mjcf_model(model)

First of all, the environment must be verified by rendering a picture.

In [ ]:
mediapy.show_image(physics.render(height, width, camera_id=0))

Define IDs for two objects in order to get timestamp of collision

In [ ]:
box1_id = physics.model.name2id("left_box/box_geom", "geom")
box2_id = physics.model.name2id("right_box/box_geom", "geom")
box1_id, box2_id

Next, it's time to make a simulation. This might take some time.

In [ ]:
physics.reset()

frames = []
timevals = []
velocity = []
position = []
collisions = []

accelerate = None
while physics.data.time < duration:
    if physics.data.time >= 1.0 and not accelerate:
        physics.data.qvel = [left_velocity, 0, -right_velocity, 0]
        accelerate =True

    timevals.append(physics.data.time)
    velocity.append([physics.data.qvel[0], physics.data.qvel[2]])
    position.append(
        [
            physics.named.data.geom_xpos["left_box/box_geom"][0],
            physics.named.data.geom_xpos["right_box/box_geom"][0],
        ]
    )

    for i, c in enumerate(physics.data.contact):
        if box1_id in c.geom and box2_id in c.geom:
            collisions.append((c, physics.data.time))

    if len(frames) < physics.data.time * framerate:
        pixels = physics.render(height, width, camera_id=0)
        frames.append(pixels)
    
    physics.step()

In [ ]:
mediapy.show_video(frames, fps=framerate)

Save the rendered video

In [ ]:
collision_type = "inelastic" if is_inelastic else "elastic"
mass_type = "same-mass"
if left_mass != right_mass:
    mass_type = "blue" if left_mass > right_mass else "red"
    mass_type = f"{mass_type}-bigger"

video_name = f"collision-{collision_type}-{mass_type}"
if not IS_COLAB:
    video_name = f"../../output/" + video_name
mediapy.write_video(video_name + ".mp4", images=frames, fps=framerate)

## Simulation Data Visualization

Convert data into numpy array to have more features

In [ ]:
# convert velocity into numpy array to have more features
velocity = np.array(velocity)
# we split two signals into two arrays
velocity_r = velocity[:, 0]
velocity_l = velocity[:, 1]
# this line checks whether the shape of data equals, meaning that we don't lose anything
velocity_r.shape[0] == velocity_l.shape[0] == velocity.shape[0]

In [ ]:
# convert position into numpy array to have more features
position = np.array(position)
# we split two signals into two arrays
position_r = position[:, 0]
position_l = position[:, 1]
# this line checks whether the shape of data equals, meaning that we don't lose anything
position_r.shape[0] == position_l.shape[0] == position.shape[0]

## Simulation Data Visualization

Collected velocities and position now can be plotted to investigate what happened to the object.

In [ ]:
cm = 1 / 2.54
figsize = (8.5 * cm, 3.75 * cm)
fig, ax = plt.subplots(ncols=2, figsize=figsize, dpi=dpi)
fig.subplots_adjust(wspace=1 * cm)

y_labels = ["Amplitude [m/s]", "Range, [m]"]
x_labels = ["Time, [s]", "Time, [s]"]
titles = ["Velocity", "Position"]
y_data = [[velocity_r, velocity_l], [position_r, position_l]]

for i in range(2):
    sns.lineplot(x=timevals, y=y_data[i][0], ax=ax[i], linewidth=1, label=f"Blue {physics.model.body('left_box/box').mass[0]}kg", color="blue")
    sns.lineplot(x=timevals, y=y_data[i][1], ax=ax[i], linewidth=1, label=f"Red {physics.model.body('right_box/box').mass[0]}kg", color="red")
    ax[i].set_title(titles[i], fontsize=8)
    ax[i].set_ylabel(y_labels[i], fontsize=7, labelpad=2)
    ax[i].set_xlabel(x_labels[i], fontsize=7, labelpad=2)
    ax[i].tick_params(labelsize=6)
    ax[i].tick_params(which="both", direction="in", top=True, right=True, length=2)
    ax[i].legend(fontsize=5, handlelength=1).get_frame().set_linewidth(0.5)

ax[0].set_ylim(bottom=-0.1, top=1.3)

chart_name = f"collision-{collision_type}-{mass_type}"
if not IS_COLAB:
    chart_name = f"../../output/" + chart_name
fig.savefig(chart_name + ".png", bbox_inches="tight")